# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846856494448                   -0.70    4.8
  2   -7.852323051381       -2.26       -1.53    1.0
  3   -7.852618291718       -3.53       -2.55    1.5
  4   -7.852646040039       -4.56       -2.92    2.2
  5   -7.852646557553       -6.29       -3.28    1.2
  6   -7.852646679944       -6.91       -4.10    1.0
  7   -7.852646686487       -8.18       -5.23    2.0
  8   -7.852646686727       -9.62       -5.61    2.0
  9   -7.852646686730      -11.51       -6.75    1.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846855401655                   -0.70           4.5
  2   -7.852527529342       -2.25       -1.63   0.80    2.0
  3   -7.852637005760       -3.96       -2.71   0.80    1.0
  4   -7.852646475908       -5.02       -3.29   0.80    2.2
  5   -7.852646678273       -6.69       -4.14   0.80    1.2
  6   -7.852646686417       -8.09       -4.80   0.80    1.5
  7   -7.852646686723       -9.51       -5.61   0.80    2.0
  8   -7.852646686730      -11.18       -6.65   0.80    1.8


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.349863e+01     3.424852e+00
 * time: 0.5210709571838379
     1     1.068269e+00     1.892484e+00
 * time: 0.800713062286377
     2    -1.337950e+00     2.004063e+00
 * time: 0.8372750282287598
     3    -3.739882e+00     1.843201e+00
 * time: 0.8889491558074951
     4    -4.993274e+00     1.742647e+00
 * time: 0.9408040046691895
     5    -6.768633e+00     1.007818e+00
 * time: 0.9945461750030518
     6    -7.417112e+00     5.816504e-01
 * time: 1.0459771156311035
     7    -7.684623e+00     1.718047e-01
 * time: 1.0811591148376465
     8    -7.776602e+00     1.725972e-01
 * time: 1.1177480220794678
     9    -7.813051e+00     8.594681e-02
 * time: 1.1542229652404785
    10    -7.832713e+00     6.826463e-02
 * time: 1.1907501220703125
    11    -7.842445e+00     7.037281e-02
 * time: 1.2263751029968262
    12    -7.846136e+00     5.091066e-02
 * time: 1.2629220485687256
    13    -7.848485e+00     2.888792e-02
 * time: 1.29915809631

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846840489541                   -0.70    4.5


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852645904244                   -1.64
  2   -7.852646686730       -6.11       -3.71
  3   -7.852646686730      -13.30       -7.24


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.8378812444638547e-7
|ρ_newton - ρ_scfv| = 1.0063797986935172e-7
|ρ_newton - ρ_dm|   = 6.357634850034107e-10
